In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
ed = tfp.edward2
import numpy as np

from future_features import tape
from mapper import Mapper

from testsuite import *

/home/folzd/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


ImportError: cannot import name 'independentFactorAnalysisTest'

In [ ]:
N = 10000
Ntest = 10000

n_features = 2
n_restarts = 1
n_datasets = 1
deviations = np.logspace(-3,1,5, dtype='float32')

In [ ]:
deviations = deviations[-1:] # take a large-variance prior on the cluster means, i.e. spread-out clusters, i.e. high snr

In [ ]:
with tape() as reference_tf:
    data_tf = mixtureOfGaussians(n_observations=N + Ntest, n_components=2, n_features=n_features, mixture_component_means_std=placeholder_deviation)
        
with tf.Session() as sess:
    for deviation in deviations:
        for dataset in range(n_datasets):
            data, reference = sess.run([data_tf, reference_tf], feed_dict={placeholder_deviation: deviation})
            loss = {}
            opt = {}
            for model in models: 
                loss[model.model_name], opt[model.model_name] = model.map_optimizer(data=data[:N])
            for restart in range(n_restarts):        
                sess.run(tf.global_variables_initializer())
                for i,model in enumerate(models): 
                    opt[model.model_name].minimize()
                    MAP_parameter, converged_loss = sess.run([model.variables, loss[model.model_name]])
                    MAP_parameters[(model.model_name, deviation, restart, dataset)] = MAP_parameter
                    train_neg_log_joint.loc[{'model': model.model_name, 'deviation': deviation, 'restart': restart, 'dataset': dataset}] = converged_loss
                    train_neg_log_lik.loc[{'model': model.model_name, 'deviation': deviation, 'restart': restart, 'dataset': dataset}] = neg_log_lik(MAP_parameter,test_models[i],sess,data[:N])
                    test_neg_log_lik.loc[{'model': model.model_name, 'deviation': deviation, 'restart': restart, 'dataset': dataset}] = neg_log_lik(MAP_parameter,test_models[i],sess,data[N:])